<h1> <center> 8 - Group vs Nationality </center> </h1>

**Author:** Sebastião Rosalino, nº 20230372

**Supervisors:** Flávio Pinheiro, António Curado

<div style="text-align: center;">
    <img src="../logos/nova_ims_logo.png" alt="Image 1" style="width: 15%; display: inline-block; margin: 0 10px;"/>
    <img src="../logos/european_parliament_logo.png" alt="Image 2" style="width: 15%; display: inline-block; margin: 0 10px;"/>
</div>

# Table of Contents

1. [Data Preparation](#Data-Preparation)
2. [EP Group vs Nationality Index](#EP-Group-vs-Nationality-Index)
   - 2.1. [All subjects](#All-subjects)
   - 2.2. [European citizenship](#European-citizenship)
   - 2.3. [Internal market, single market](#Internal-market-single-market)
   - 2.4. [Community policies](#Community-policies)
   - 2.5. [Economic, social and territorial cohesion](#Economic-social-and-territorial-cohesion)
   - 2.6. [Economic and monetary system](#Economic-and-monetary-system)
   - 2.7. [External relations of the Union](#External-relations-of-the-Union)
   - 2.8. [Area of freedom, security and justice](#Area-of-freedom-security-and-justice)
   - 2.9. [State and evolution of the Union](#State-and-evolution-of-the-Union)
   - 2.10. [Secondary Subjects](#Secondary-Subjects)
   - 2.11. [Tertiary Subjects](#Tertiary-Subjects)

In [1]:
# Import the necessary libraries and configurations
import pandas as pd
import warnings
import numpy as np

from datetime import datetime
from tqdm import tqdm
from scipy.stats import ttest_1samp

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
# Read all the datasets from their corresponding folders
group_memberships = pd.read_csv("../datasets/second_stage/3_group_memberships/group_memberships.csv")
bills = pd.read_csv("../datasets/second_stage/4_bills/bills.csv")
member_votes = pd.read_csv("../datasets/second_stage/5_member_votes/member_votes.csv")
primary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/primary_subject_bills.csv")
secondary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/secondary_subject_bills.csv")
tertiary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/tertiary_subject_bills.csv")

In [3]:
# A reminder of how the group_memberships dataset looks like
group_memberships

,group_code,start_date,end_date,id_mep,membership_length
0,RENEW,2019-07-02,2024-07-15,0,1840
1,RENEW,2019-07-02,2020-01-31,1,213
2,SD,2019-07-02,2022-10-02,2,1188
3,EPP,2023-07-07,2024-07-15,3,374
4,SD,2019-07-02,2024-07-15,4,1840
...,...,...,...,...,...
964,EPP,2024-03-26,2024-07-15,868,111
965,EPP,2024-04-02,2024-07-15,869,104
966,EPP,2024-04-02,2024-07-15,870,104
967,EPP,2024-04-02,2024-07-15,871,104


In [4]:
# Convert 'start_date' and 'end_date' columns to datetime
group_memberships['start_date'] = pd.to_datetime(group_memberships['start_date'])
group_memberships['end_date'] = pd.to_datetime(group_memberships['end_date'])

In [5]:
# A reminder of how the bills dataset looks like
bills

,timestamp,procedure_reference,id_bill,text
0,2020-06-17 23:24:52,2007/0181(NLE),200,ecmorocco euro-mediterranean aviation agreemen...
1,2021-12-14 14:49:19,2010/0112(NLE),786,recommendation draft council decision conclusi...
2,2020-06-17 23:24:52,2010/0180(NLE),201,eujordan agreement euro-mediterranean aviation...
3,2020-06-17 23:24:52,2010/0186(NLE),203,eugeorgia agreement common aviation area
4,2020-06-17 23:24:52,2012/0006(NLE),198,eumoldova common aviation area agreement
...,...,...,...,...
1898,2024-04-25 12:51:39,2024/2696(RSP),1902,joint motion resolution new allegation russian...
1899,2024-04-25 12:20:42,2024/2698(RSP),1898,european parliament resolution azerbaijan nota...
1900,2024-04-25 12:21:01,2024/2699(RSP),1901,european parliament resolution propose repeal ...
1901,2024-04-25 12:53:44,2024/2703(RSP),1900,joint motion resolution


In [6]:
# A reminder of how the member_votes dataset looks like
member_votes

,position,country_code,group_code,id_mep,id_bill
0,FOR,LUX,RENEW,0,0
1,FOR,GBR,RENEW,1,0
2,DID_NOT_VOTE,DEU,SD,2,0
3,FOR,DEU,SD,4,0
4,FOR,DEU,SD,5,0
...,...,...,...,...,...
1345669,DID_NOT_VOTE,PRT,EPP,868,1902
1345670,DID_NOT_VOTE,PRT,EPP,869,1902
1345671,DID_NOT_VOTE,PRT,EPP,870,1902
1345672,FOR,PRT,EPP,871,1902


## 1. Data Preparation
<a id="Data-Preparation"></a>

In [7]:
# Replace values in the 'position' column as follows:
# FOR for "supporting"
# AGAINST or ABSTENTION or DID_NOT_VOTE for "not_supporting" 
member_votes['position'] = member_votes['position'].replace({
    "FOR": "supporting",
    "AGAINST": "not_supporting",
    "ABSTENTION": "not_supporting",
    "DID_NOT_VOTE": "not_supporting"
})

# Visualize the updated dataset
member_votes

,position,country_code,group_code,id_mep,id_bill
0,supporting,LUX,RENEW,0,0
1,supporting,GBR,RENEW,1,0
2,not_supporting,DEU,SD,2,0
3,supporting,DEU,SD,4,0
4,supporting,DEU,SD,5,0
...,...,...,...,...,...
1345669,not_supporting,PRT,EPP,868,1902
1345670,not_supporting,PRT,EPP,869,1902
1345671,not_supporting,PRT,EPP,870,1902
1345672,supporting,PRT,EPP,871,1902


## 2. EP Group vs Nationality Index
<a id="EP-Group-vs-Nationality-Index"></a>

### **EP Group vs Nationality Index**

##### The EP Group vs Nationality Index measures the degree to which the voting behavior of MEPs from a given country aligns with ideological expectations based on EP representation. It provides insight into the influence of national interests versus EP representation in bills.
------------------------------------

1. **Observed Voting Behavior**:
   - For each country $c$, the observed voting behavior on a bill is summarized using the **Observed Rice Index**, defined as:
     $$
     R^{\text{obs}}_c = \frac{|S_c - N_c|}{S_c + N_c}
     $$
     where:
     - $S_c$ is the number of MEPs from country $c$ who voted in support of the bill.
     - $N_c$ is the number of MEPs from country $c$ who did not support the bill.
     - $S_c + N_c$ is the total number of MEPs from country $c$ who participated in the vote of the bill.

<br>

2. **Expected Voting Behavior**:
   - The expected voting behavior on a bill is calculated based on the proportional representation of political groups in the EP when the bill was being voted. Let $p_g$ be the proportion of MEPs in group $g$ when the bill was being voted, defined as:
     $$
     p_g = \frac{\text{Number of MEPs in group } g \text{ when the bill was being voted}}{\text{Total number of MEPs in the EP when the bill was being voted}}
     $$
   - Using these proportions, 100 MEPs are randomly sampled across all groups. Their voting positions are used to compute the **Expected Rice Index**:
     $$
     R^{\text{exp}} = \frac{|S^{\text{exp}} - N^{\text{exp}}|}{S^{\text{exp}} + N^{\text{exp}}}
     $$
     where $S^{\text{exp}}$ and $N^{\text{exp}}$ are the counts of supporting and opposing votes, respectively.

<br>

3. **Difference Calculation**:
   - The deviation for country $c$ in a bill is calculated as the difference between **observed** and **expected** **Rice Index**:
     $$
     \Delta R_c = R^{\text{obs}}_c - R^{\text{exp}}
     $$

4. **Aggregation**:
   - The deviations $\Delta R_c$ are averaged across all bills $B$ to compute the **EP Group vs Nationality Index** for each country:
     $$
     I_c = \frac{1}{B} \sum_{b=1}^{B} \Delta R_{c,b}
     $$
     where $\Delta R_{c,b}$ is the difference calculated for country $c$ on bill $b$, and $B$ is the total number of bills considered.

<br>


### **Statistical Test Procedure**

To assess whether $I_c$ (the **EP Group vs Nationality Index** for a given country) is statistically different from 0, a **two-tailed one-sample $t$-test** is performed:

1. **Null Hypothesis ($H_0$)**:
   - The mean difference $\mu_{\Delta R_c} = 0$: There is no significant deviation in voting behavior alignment for the country compared to the EP.

<br>

2. **Alternative Hypothesis ($H_1$)**:
   - The mean difference $\mu_{\Delta R_c} \neq 0$: MEPs from the country exhibit a voting behavior alignment that significantly differs from what would be expected based on EP group representation.

<br>

3. **Test Statistic**:
   - The $t$-statistic is calculated as:
     $$
     t = \frac{\overline{\Delta R_c}}{\frac{s_{\Delta R_c}}{\sqrt{n}}}
     $$
     where:
     - $\overline{\Delta R_c}$: Mean of the differences for the country.
     - $s_{\Delta R_c}$: Sample standard deviation of the differences.
     - $n$: Number of bills considered for the country.

<br>

4. **Significance Level**:
   - A **two-tailed** $p$-value is computed to test whether the mean is **significantly different** from 0.
   - Results are considered statistically significant if **$p < 0.05$**.

<br>

5. **Filtering Statistically Significant Results**:
   - Only countries with **$p < 0.05$** are considered in the results. This ensures that observed deviations are unlikely to have occurred by chance.

<br>

### Interpretation

The **EP Group vs Nationality Index** $I_c$ provides insight into the cohesiveness of voting behavior among MEPs from a given country compared to the EP as a whole:
- A **positive** $I_c$ indicates that MEPs from a given country exhibit greater cohesiveness in their voting behavior than the EP as a whole.
- A **negative** $I_c$ indicates that MEPs from a given country exhibit less cohesiveness in their voting behavior compared to the EP as a whole.

In [8]:
def calculate_rice_index(positions):
    """Calculate the Rice Index for a given set of positions."""
    supporting = positions.count("supporting")
    not_supporting = positions.count("not_supporting")
    total = len(positions)
    if total == 0:
        return 0
    return abs(supporting - not_supporting) / total


def compute_ep_group_vs_nationality_index(bills):

    # Skip the execution of the EP Group vs Nationality Index if there aren't at least 30 bills
    if len(bills) < 30:
        return None

    def compute_index():

        # Set a seed for reproducibility
        np.random.seed(22)
    
        country_results = []

        # Iterate over each bill
        for _, bill in tqdm(bills.iterrows(), total=len(bills), desc="Processing bills"):
            
            timestamp = datetime.strptime(bill['timestamp'], "%Y-%m-%d %H:%M:%S")
            
            id_bill = bill['id_bill']

            # Filter for active group memberships at the bill's timestamp
            active_memberships = group_memberships[(group_memberships['start_date'] <= timestamp) & 
                                                   (group_memberships['end_date'] >= timestamp)]

            # Calculate group representation (including the NI MEPs, as for this method, the NI set of MEPs are considered to be part of the "NI group") percentages
            group_counts = active_memberships['group_code'].value_counts()
            total_meps = group_counts.sum()
            group_probabilities = group_counts / total_meps

            # Filter for votes for the current bill
            bill_votes = member_votes[member_votes['id_bill'] == id_bill]

            # Calculate the Rice indices (Observed and Expected) for each country
            for country, group in bill_votes.groupby('country_code'):
                
                # Observed Rice Index of all MEPs from the country who participated in the bill
                observed_rice_index = calculate_rice_index(group['position'].tolist())

                # Calculate the Expected Rice Index by sampling 100 random MEPs from all groups (including the "NI group") and calculating the Rice Index
                simulated_positions = []
                for _ in range(100):
                    sampled_group = np.random.choice(group_probabilities.index, p=group_probabilities.values)
                    sampled_votes = bill_votes[bill_votes['group_code'] == sampled_group]
                    if not sampled_votes.empty:
                        simulated_positions.append(np.random.choice(sampled_votes['position'].tolist()))

                expected_rice_index = calculate_rice_index(simulated_positions)

                # Subtract the Expected Rice Index from the Observed Rice Index. Negative values imply that Group alignment plays a stronger role than National alignment,
                # in the country's MEPs votes. Positive values imply that National alignment plays a stronger role than Group alignment, in the country's MEP votes
                difference = observed_rice_index - expected_rice_index
                country_results.append((country, difference))

        # Convert country_results to a pandas dataframe
        country_df = pd.DataFrame(country_results, columns=['country', 'difference'])

        # Perform two-tailed t-tests to test if the differences are significantly different from 0 for each country
        test_results = []
        for country, group in country_df.groupby('country'):
            differences = group['difference'].values
            t_stat, p_value = ttest_1samp(differences, 0, alternative='two-sided')
            test_results.append({
                'Country': country,
                'EP Group vs Nationality Index': differences.mean(),
                'Significant': p_value < 0.05  # Mark as statistically significant if p-value < 0.05
            })

        # Convert test_results to a pandas dataframe
        test_results_df = pd.DataFrame(test_results)

        # Filter for statistically significant results only
        test_results_df = test_results_df[test_results_df['Significant']]

        # Drop the 'Significant' column
        test_results_df = test_results_df.drop(columns=['Significant'])

        # Sort by the EP Group vs Nationality Index in descending order
        test_results_df = test_results_df.sort_values(by='EP Group vs Nationality Index', ascending=False).reset_index(drop=True)

        return test_results_df

    # Compute the EP Group vs Nationality Index with statistical tests
    return compute_index()

### 2.1. All subjects
<a id="All-subjects"></a>

In [9]:
# Compute the EP Group vs Nationality Index for all subjects, regardless of primary subject
all_subjects_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(bills)

all_subjects_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 1903/1903 [29:32<00:00,  1.07it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.306810
1,SVN,0.278807
2,PRT,0.165141
3,LTU,0.162593
4,MLT,0.160599
5,IRL,0.158365
6,EST,0.152266
7,CYP,0.150440
8,ROU,0.142494
9,ESP,0.138084


### 2.2. European citizenship
<a id="European-citizenship"></a>

In [10]:
# Filter for bills whose primary subject is 1 - European citizenship (among other potential primary subjects)
primary_subject_bills_1 = primary_subject_bills[primary_subject_bills["primary_subject"] == "European citizenship"]

In [11]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 1 (among other potential primary subjects)
primary_subject_1_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_1)

primary_subject_1_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 98/98 [01:32<00:00,  1.06it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.335510
1,IRL,0.249647
2,EST,0.216045
3,CYP,0.190816
4,SVN,0.182041
5,PRT,0.180991
6,DNK,0.168648
7,SWE,0.144490
8,ESP,0.141826
9,FIN,0.126829


### 2.3. Internal market, single market
<a id="Internal-market-single-market"></a>

In [12]:
# Filter for bills whose primary subject is 2 - Internal market, single market (among other potential primary subjects)
primary_subject_bills_2 = primary_subject_bills[primary_subject_bills["primary_subject"] == "Internal market, single market"]

In [13]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 2 (among other potential primary subjects)
primary_subject_2_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_2)

primary_subject_2_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 195/195 [03:04<00:00,  1.05it/s]


,Country,EP Group vs Nationality Index
0,SVN,0.347538
1,LUX,0.259521
2,EST,0.173294
3,LTU,0.161091
4,ROU,0.156310
5,MLT,0.150359
6,BGR,0.145140
7,ESP,0.133367
8,IRL,0.113567
9,DNK,0.112584


### 2.4. Community policies
<a id="Community-policies"></a>

In [14]:
# Filter for bills whose primary subject is 3 - Community policies (among other potential primary subjects)
primary_subject_bills_3 = primary_subject_bills[primary_subject_bills["primary_subject"] == "Community policies"]

In [15]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 3 (among other potential primary subjects)
primary_subject_3_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_3)

primary_subject_3_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 597/597 [09:35<00:00,  1.04it/s]


,Country,EP Group vs Nationality Index
0,SVN,0.302529
1,LUX,0.250262
2,CYP,0.181295
3,MLT,0.174662
4,EST,0.154089
5,PRT,0.143510
6,LTU,0.135249
7,ROU,0.135243
8,BGR,0.133260
9,IRL,0.129949


### 2.5. Economic, social and territorial cohesion
<a id="Economic-social-and-territorial-cohesion"></a>

In [16]:
# Filter for bills whose primary subject is 4 - Economic, social and territorial cohesion (among other potential primary subjects)
primary_subject_bills_4 = primary_subject_bills[primary_subject_bills["primary_subject"] == "Economic, social and territorial cohesion"]

In [17]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 4 (among other potential primary subjects)
primary_subject_4_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_4)

primary_subject_4_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 394/394 [06:12<00:00,  1.06it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.313519
1,IRL,0.279395
2,SVN,0.245812
3,CYP,0.241591
4,PRT,0.221375
5,MLT,0.176277
6,ESP,0.160731
7,LTU,0.148708
8,EST,0.144498
9,ROU,0.144399


### 2.6. Economic and monetary system
<a id="Economic-and-monetary-system"></a>

In [18]:
# Filter for bills whose primary subject is 5 - Economic and monetary system (among other potential primary subjects)
primary_subject_bills_5 = primary_subject_bills[primary_subject_bills["primary_subject"] == "Economic and monetary system"]

In [19]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 5 (among other potential primary subjects)
primary_subject_5_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_5)

primary_subject_5_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 32/32 [00:30<00:00,  1.04it/s]


,Country,EP Group vs Nationality Index
0,SVN,0.435000
1,LUX,0.364167
2,BGR,0.216691
3,LTU,0.211477
4,ROU,0.207749
5,PRT,0.205060
6,ESP,0.181791
7,MLT,0.178125
8,HRV,0.168371
9,HUN,0.167351


### 2.7. External relations of the Union
<a id="External-relations-of-the-Union"></a>

In [20]:
# Filter for bills whose primary subject is 6 - External relations of the Union (among other potential primary subjects)
primary_subject_bills_6 = primary_subject_bills[primary_subject_bills["primary_subject"] == "External relations of the Union"]

In [21]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 6 (among other potential primary subjects)
primary_subject_6_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_6)

primary_subject_6_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 444/444 [07:01<00:00,  1.05it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.324219
1,SVN,0.292905
2,EST,0.184719
3,SWE,0.177711
4,POL,0.160834
5,ROU,0.160457
6,LTU,0.158731
7,DNK,0.154587
8,PRT,0.131553
9,MLT,0.128363


### 2.8. Area of freedom, security and justice
<a id="Area-of-freedom-security-and-justice"></a>

In [22]:
# Filter for bills whose primary subject is 7 - Area of freedom, security and justice (among other potential primary subjects)
primary_subject_bills_7 = primary_subject_bills[primary_subject_bills["primary_subject"] == "Area of freedom, security and justice"]

In [23]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 7 (among other potential primary subjects)
primary_subject_7_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_7)

primary_subject_7_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 181/181 [02:52<00:00,  1.05it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.320921
1,SVN,0.235414
2,EST,0.194007
3,LTU,0.152938
4,MLT,0.146409
5,PRT,0.146272
6,POL,0.139675
7,SWE,0.137032
8,ESP,0.134980
9,DNK,0.128998


### 2.9. State and evolution of the Union
<a id="State-and-evolution-of-the-Union"></a>

In [24]:
# Filter for bills whose primary subject is 8 - State and evolution of the Union (among other potential primary subjects)
primary_subject_bills_8 = primary_subject_bills[primary_subject_bills["primary_subject"] == "State and evolution of the Union"]

In [25]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 8 (among other potential primary subjects)
primary_subject_8_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_8)

primary_subject_8_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 569/569 [08:57<00:00,  1.06it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.322835
1,SVN,0.264341
2,IRL,0.224960
3,LTU,0.201882
4,LVA,0.183866
5,PRT,0.177168
6,CYP,0.175653
7,ESP,0.165375
8,DNK,0.157619
9,MLT,0.137868


### 2.10. Secondary Subjects
<a id="Secondary-Subjects"></a>

In [26]:
# Iterate over each unique secondary subject and calculate the EP Country vs Nationality Index for each secondary subject
for secondary_subject in secondary_subject_bills['secondary_subject'].unique():
    print(f"Secondary subject: {secondary_subject}")
    subject_bills = secondary_subject_bills[secondary_subject_bills['secondary_subject'] == secondary_subject]
    results = compute_ep_group_vs_nationality_index(subject_bills)
    print(results)
    print('\n')
    print("--------------------")
    print('\n')

Secondary subject: Transport policy in general


Processing bills: 100%|██████████| 89/89 [01:25<00:00,  1.04it/s]


   Country  EP Group vs Nationality Index
0      SVN                       0.238427
1      LUX                       0.210712
2      EST                       0.140717
3      BGR                       0.138493
4      IRL                       0.135839
5      ESP                       0.112434
6      ROU                       0.108742
7      MLT                       0.091685
8      AUT                       0.077469
9      POL                       0.076400
10     LTU                       0.072666
11     PRT                       0.066089
12     CYP                       0.065843
13     DNK                       0.064215
14     SWE                       0.062076
15     NLD                       0.048999
16     BEL                       0.041552
17     DEU                       0.035518
18     GRC                      -0.152413


--------------------


Secondary subject: Relations with third countries


Processing bills: 100%|██████████| 116/116 [01:49<00:00,  1.06it/s]


   Country  EP Group vs Nationality Index
0      LUX                       0.373506
1      SVN                       0.311897
2      EST                       0.200189
3      ROU                       0.165797
4      SWE                       0.154639
5      LTU                       0.154075
6      POL                       0.149837
7      DNK                       0.128799
8      ESP                       0.128541
9      HUN                       0.128013
10     MLT                       0.122586
11     PRT                       0.122217
12     BGR                       0.116105
13     LVA                       0.088362
14     HRV                       0.085789
15     AUT                       0.074589
16     FIN                       0.070964
17     IRL                       0.063130
18     SVK                       0.059074
19     CZE                       0.050402
20     NLD                       0.047379
21     FRA                      -0.142466
22     GRC                      -0

...

   Country  EP Group vs Nationality Index
0      IRL                       0.393090
1      CYP                       0.353153
2      LUX                       0.301802
3      PRT                       0.280669
4      LTU                       0.215528
5      ROU                       0.202748
6      SVN                       0.198108
7      LVA                       0.172432
8      MLT                       0.164505
9      ESP                       0.155158
10     BGR                       0.121463
11     EST                       0.092252
12     HRV                       0.084439
13     AUT                       0.052170
14     NLD                      -0.086426
15     CZE                      -0.090579
16     GRC                      -0.114826
17     ITA                      -0.143840


--------------------


Secondary subject: Citizen's rights


Processing bills: 100%|██████████| 68/68 [01:05<00:00,  1.03it/s]


   Country  EP Group vs Nationality Index
0      LUX                       0.283039
1      EST                       0.223711
2      SVN                       0.222941
3      IRL                       0.176166
4      DNK                       0.173681
5      PRT                       0.143403
6      LTU                       0.122139
7      ESP                       0.119535
8      SWE                       0.114678
9      CYP                       0.107059
10     HRV                       0.100847
11     ROU                       0.085357
12     AUT                       0.081987
13     FIN                       0.077195
14     MLT                       0.072941
15     GRC                      -0.150532
16     GBR                      -0.578777


--------------------


Secondary subject: Competition
None


--------------------


Secondary subject: Emergency, food, humanitarian aid
None


--------------------


Secondary subject: Fundamental rights in the EU, Charter
None


-----------

Processing bills: 100%|██████████| 69/69 [01:06<00:00,  1.03it/s]

   Country  EP Group vs Nationality Index
0      LUX                       0.271981
1      SVN                       0.262754
2      MLT                       0.219420
3      IRL                       0.213222
4      LVA                       0.197826
5      PRT                       0.180386
6      CYP                       0.179517
7      ESP                       0.153001
8      ROU                       0.136608
9      BGR                       0.136010
10     LTU                       0.135310
11     EST                       0.134617
12     HUN                       0.112602
13     GRC                      -0.092174


--------------------


Secondary subject: Tourism
None


--------------------


Secondary subject: Civil protection
None


--------------------


Secondary subject: Free movement of workers
None


--------------------


Secondary subject: Economic union
None


--------------------


Secondary subject: Economic growth
None


--------------------


Secondary subject: 

### 2.11. Tertiary Subjects
<a id="Tertiary-Subjects"></a>

In [27]:
# Iterate over each unique tertiary subject and calculate the EP Country vs Nationality Index for each tertiary subject
for tertiary_subject in tertiary_subject_bills['tertiary_subject'].unique():
    print(f"Tertiary subject: {tertiary_subject}")
    subject_bills = tertiary_subject_bills[tertiary_subject_bills['tertiary_subject'] == tertiary_subject]
    results = compute_ep_group_vs_nationality_index(subject_bills)
    print(results)
    print('\n')
    print("--------------------")
    print('\n')

Tertiary subject: Transport agreements and cooperation
None


--------------------


Tertiary subject: Relations with the Mediterranean and southern European countries
None


--------------------


Tertiary subject: Public procurement
None


--------------------


Tertiary subject: Agreements and relations in the context of the World Trade Organization (WTO)
None


--------------------


Tertiary subject: Export/import control, trade defence, trade barriers
None


--------------------


Tertiary subject: Bilateral economic and trade agreements and relations


Processing bills: 100%|██████████| 59/59 [00:57<00:00,  1.02it/s]


   Country  EP Group vs Nationality Index
0      SVN                       0.436780
1      LUX                       0.332203
2      EST                       0.309895
3      POL                       0.205737
4      LTU                       0.191371
5      SWE                       0.179952
6      DNK                       0.176934
7      ROU                       0.164810
8      SVK                       0.159348
9      BGR                       0.147019
10     LVA                       0.120339
11     MLT                       0.109379
12     HRV                       0.105074
13     AUT                       0.098242
14     CZE                       0.093430
15     HUN                       0.093220
16     ESP                       0.090526
17     PRT                       0.084843
18     FRA                      -0.095196
19     GRC                      -0.161033


--------------------


Tertiary subject: Union Customs Code, tariffs, preferential arrangements, rules of origin
Non

Processing bills: 100%|██████████| 30/30 [00:29<00:00,  1.03it/s]


   Country  EP Group vs Nationality Index
0      LUX                       0.360000
1      EST                       0.280190
2      POL                       0.277746
3      MLT                       0.269556
4      HUN                       0.239968
5      PRT                       0.231778
6      SVN                       0.231333
7      DNK                       0.207648
8      BGR                       0.194275
9      LTU                       0.186303
10     CZE                       0.184698
11     ESP                       0.164967
12     CYP                       0.141333
13     SWE                       0.138222
14     LVA                       0.132000
15     ROU                       0.129833
16     SVK                       0.122601
17     NLD                       0.118821
18     AUT                       0.116869
19     FIN                       0.079187


--------------------


Tertiary subject: Agencies and bodies of the EU
None


--------------------


Tertiary subjec

...

   Country  EP Group vs Nationality Index
0      CYP                       0.527725
1      FRA                       0.424719
2      HUN                       0.348133
3      MLT                       0.339259
4      SVN                       0.322222
5      AUT                       0.319959
6      HRV                       0.316479
7      ITA                       0.242660
8      GRC                       0.229690
9      LVA                       0.225873
10     SVK                       0.194192
11     POL                       0.143102
12     BGR                       0.136527
13     PRT                       0.120438
14     LUX                       0.113757
15     ESP                      -0.071198
16     NLD                      -0.110409
17     DEU                      -0.135236


--------------------


Tertiary subject: Dangerous substances, toxic and radioactive wastes (storage, transport)
None


--------------------


Tertiary subject: Health services, medical institutions
N

Processing bills: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


   Country  EP Group vs Nationality Index
0      SVN                       0.415250
1      LUX                       0.292167
2      EST                       0.232429
3      MLT                       0.228333
4      POL                       0.208972
5      ROU                       0.207091
6      BGR                       0.205265
7      HUN                       0.203286
8      ESP                       0.118779
9      NLD                       0.118500
10     DNK                       0.116857
11     SVK                       0.106857
12     LTU                       0.105273
13     LVA                       0.094250
14     GRC                      -0.165786


--------------------


Tertiary subject: Insurance, pension funds
None


--------------------


Tertiary subject: Medicine, diseases


Processing bills: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]

   Country  EP Group vs Nationality Index
0      LUX                       0.331579
1      MLT                       0.297368
2      SVN                       0.247632
3      CYP                       0.215789
4      PRT                       0.200652
5      LVA                       0.200000
6      ROU                       0.195024
7      BGR                       0.168142
8      IRL                       0.167733
9      HUN                       0.164887
10     ESP                       0.162720
11     EST                       0.146767
12     LTU                       0.145933
13     ITA                      -0.124054


--------------------


Tertiary subject: Business policy, e-commerce, after-sales service, commercial distribution
None


--------------------


Tertiary subject: Youth
None


--------------------


Tertiary subject: Man-made disasters, industrial pollution and accidents
None


--------------------


Tertiary subject: Peace preservation, humanitarian and rescue task